In [ ]:
# Import libraries
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
plt.style.use('ggplot')

import ast
import contractions
from wordcloud import WordCloud
import emoji
import re
import string 
import nltk
nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from collections import Counter
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer
from tqdm.notebook import tqdm 
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
# Reload the Data
df = pd.read_csv('dataset/afterEDA_DatingAppReviewsDataset.csv')
df.head()
df.info()

,Id,neg,neu,pos,compound,Name,Review,Date&Time,App,Sentiment,Tokens,Sentiment_Numeric,TokenCount
0,111028,0.074,0.926,0.000,-0.0516,Jasper Ancajas,i have a hard time logging in so i uninstalled...,2020-05-28 19:28:00,Tinder,negative,"['hard', 'time', 'log', 'uninstalled', 'app', ...",-1,9
1,211245,0.167,0.833,0.000,-0.2960,Tiger 181,i am getting no match and like...in stead of s...,2019-03-03 21:08:00,Tinder,negative,"['get', 'no', 'match', 'like', 'stead', 'spend...",-1,8
2,9624,0.431,0.000,0.569,0.1779,Mr 21,no good,2021-02-03 21:38:00,Hinge,negative,"['no', 'good']",-1,2
3,198692,0.338,0.519,0.143,-0.4588,Tim Stone,"got banned for no reason, paid customer for ye...",2019-04-20 13:56:00,Tinder,negative,"['get', 'ban', 'no', 'reason', 'paid', 'custom...",-1,9
4,27599,0.180,0.755,0.065,-0.5057,Thelma Barbara,the more i use this app the more i am appalled...,2021-08-31 17:19:00,Tinder,negative,"['use', 'app', 'appalled', 'incredible', 'numb...",-1,6


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Id                 50000 non-null  int64  
 1   neg                50000 non-null  float64
 2   neu                50000 non-null  float64
 3   pos                50000 non-null  float64
 4   compound           50000 non-null  float64
 5   Name               50000 non-null  object 
 6   Review             49950 non-null  object 
 7   Date&Time          50000 non-null  object 
 8   App                50000 non-null  object 
 9   Sentiment          50000 non-null  object 
 10  Tokens             50000 non-null  object 
 11  Sentiment_Numeric  50000 non-null  int64  
 12  TokenCount         50000 non-null  int64  
dtypes: float64(4), int64(3), object(6)
memory usage: 5.0+ MB


## Feature Extraction 

In [10]:
# 'Tokens' column needs to be converted from string to list
df['Tokens'] = df['Tokens'].apply(ast.literal_eval) 

# The cleaned Tokens were joined back into space-separated strings for TF-IDF Vectorizer to work 
df['Cleaned_Review'] = df['Tokens'].apply(lambda x: ' '.join(x))

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Id                 50000 non-null  int64  
 1   neg                50000 non-null  float64
 2   neu                50000 non-null  float64
 3   pos                50000 non-null  float64
 4   compound           50000 non-null  float64
 5   Name               50000 non-null  object 
 6   Review             49950 non-null  object 
 7   Date&Time          50000 non-null  object 
 8   App                50000 non-null  object 
 9   Sentiment          50000 non-null  object 
 10  Tokens             50000 non-null  object 
 11  Sentiment_Numeric  50000 non-null  int64  
 12  TokenCount         50000 non-null  int64  
 13  Cleaned_Review     50000 non-null  object 
dtypes: float64(4), int64(3), object(7)
memory usage: 5.3+ MB


In [12]:
# Dropping all unnecessary columns
df.drop(columns=['neg','neu','pos','compound','Name','Sentiment','Review','Date&Time', 'App'],inplace = True)
df.head()

,Id,Tokens,Sentiment_Numeric,TokenCount,Cleaned_Review
0,111028,"[hard, time, log, uninstalled, app, not, insta...",-1,9,hard time log uninstalled app not install app ...
1,211245,"[get, no, match, like, stead, spending, nearly...",-1,8,get no match like stead spending nearly r
2,9624,"[no, good]",-1,2,no good
3,198692,"[get, ban, no, reason, paid, customer, year, m...",-1,9,get ban no reason paid customer year massive joke
4,27599,"[use, app, appalled, incredible, number, scammer]",-1,6,use app appalled incredible number scammer


In [ ]:
# df.to_csv('featured_DatingAppReviewsDataset.csv', index=False)
df.to_pickle('dataset/cleaned_reviews.pkl')